## Compass から勉強会名と参加率を拾ってみる

まず何がしたかったかというと、勉強会のタイトルを形態素解析して、その募集人数と参加者（＋参加待ち）の統計とったら、勉強会界隈のトレンドが追いやすくなるんじゃね？と思ったのが始まり。  
で、その為の第一歩として、Compass から勉強会情報を拾ってくる処理をまずは考えた。

### まずは requests でAPIを叩いてみる

COMPASS は [API仕様](https://connpass.com/about/api/) が公開されているので、そこから拾ってみる。

In [39]:
import requests

パラメータ組み立てて

In [16]:
compass_rest_api = 'https://connpass.com/api/v1/event/'
params = {"ym":"201904", "count":100, "order":3,"start":1}  # 4 月、100 件（max）、着信順、先頭から

query = '&'.join([f'{key}={params[key]}' for key in params.keys()])

投げてみる。

In [17]:
r_get = requests.get(compass_rest_api + '?' + query)

In [19]:
r_get.status_code

200

In [22]:
r_get.json()

{'results_returned': 100,
 'events': [{'event_url': 'https://aaa.connpass.com/event/124784/',
   'event_type': 'participation',
   'owner_nickname': 'Wakamatz',
   'series': {'url': 'https://aaa.connpass.com/',
    'id': 1886,
    'title': 'Mathematik und Programmierung'},
   'updated_at': '2019-03-19T00:04:09+09:00',
   'lat': '34.775820300000',
   'started_at': '2019-04-14T13:30:00+09:00',
   'hash_tag': 'functional_programming, data_structures',
   'title': '純粋関数型データ構造読書会 第23回',
   'event_id': 124784,
   'lon': '135.544884900000',
   'waiting': 0,
   'limit': 6,
   'owner_id': 3622,
   'owner_display_name': 'Wakamatz',
   'description': '<p><a href="https://www.amazon.co.jp/dp/4048930567/ref=pd_bxgy_14_img_2?_encoding=UTF8&amp;psc=1&amp;refRID=2A2ARBYZK60AWDDVAD3E" rel="nofollow">純粋関数型データ構造</a>の読書会です。</p>\n<p>次回は「第10章 データ構造ブートストラップ」p135からです。</p>',
   'address': '大阪府吹田市岸部南１丁目４－８',
   'catch': '',
   'accepted': 0,
   'ended_at': '2019-04-14T17:00:00+09:00',
   'place': '岸部市民センター 会議室'

一度に 100 件縛りがあるので、そこだけが問題だろうか？  
とりあえず 100 件でリクエストしてみて、100 件未満になるまでループしてリクエストしてみる。

### 実際に勉強会データ（名前と参加者数）を拾ってみた

尚、やってみてわかったのだけど、ソート順がデフォルト以外だと、タイムアウトでコケるようだ。  
多分インデックスが不足してるか何かしてるんだろうなーと思いつつ。  
（いや、それ以外はそこそこ高速で返ってくる事を考えれば、RDB じゃなくて Mongo とか Cassandra とか使ってそう）

In [43]:
class ClassRoom:
    def __init__(self, title, limit, accepted, waiting):
        def or_zero(v):
            if isinstance(v, int):
                return v
            return 0
        self.title = title
        self.limit = or_zero(limit)
        self.joins = or_zero(accepted) + or_zero(waiting)
        self.score = self.joins / self.limit if self.limit != 0 else 1
        
    def __str__(self):
        return f'ClassRoom({self.title}, {self.limit}, {self.joins}, {self.score})'

def convert(event):
    return ClassRoom(event['title'], event['limit'], event['accepted'], event['waiting'])


def request_to_compass(start):
    params = {"ym":"201904", "count":WANT_EVENT, "order":1,"start":start}
    query = '&'.join([f'{key}={params[key]}' for key in params.keys()])
    return requests.get(compass_rest_api + '?' + query)


WANT_EVENT = 100
getted = WANT_EVENT
current_start = 1
class_rooms = []

while getted == WANT_EVENT:
    r_get = request_to_compass(current_start)
    respond_json = r_get.json()

    getted = respond_json['results_returned']
    current_start += WANT_EVENT
    print(f'{getted}, {current_start}')

    for ev in respond_json['events']:
        class_rooms.append(convert(ev))

# least 100
r_get = r_get = request_to_compass(current_start)
respond_json = r_get.json()
for ev in respond_json['events']:
    class_rooms.append(convert(ev))

100, 101
100, 201
100, 301
100, 401
100, 501
14, 601


In [44]:
[str(s) for s in class_rooms]

['ClassRoom(もくもく執筆会☆出張版 REV.20 @ 神田神保町 〜技術系同人誌／商業誌・発表資料・ブログ〜, 0, 9, 1)',
 'ClassRoom(Azure DevOps Tokyo, Japan 4th answer, 100, 10, 0.1)',
 'ClassRoom(【CoderDojo生駒】：子ども向けプログラミング道場（No.34）, 32, 0, 0.0)',
 'ClassRoom(純粋関数型データ構造読書会 第23回, 6, 0, 0.0)',
 'ClassRoom(4/21子どもの無料プログラミングサークル【第1回CoderDojo溝口】(川崎,初心者歓迎), 10, 5, 0.5)',
 'ClassRoom(第35回 Machine Learning 15minutes!, 50, 0, 0.0)',
 'ClassRoom((若手)もくもく会＠静岡, 10, 1, 0.1)',
 'ClassRoom(Kubernetes Meetup Tokyo #18, 195, 8, 0.041025641025641026)',
 'ClassRoom(【デモ】iPadで AI:人工知能・ML:機械学習を体験, 20, 4, 0.2)',
 'ClassRoom(ぬるぽへさん快癒祝い second edition & 入稿祝い, 10, 1, 0.1)',
 'ClassRoom(Repro Tech #7 Practical AI Supported by NAVITIME, 70, 24, 0.34285714285714286)',
 'ClassRoom(Docker Workshop in Kyoto #03, 10, 2, 0.2)',
 'ClassRoom(プログラミング・エラーなんかコワくない??, 15, 4, 0.26666666666666666)',
 'ClassRoom(実践者向けディープラーニング勉強会\u3000第二回, 50, 21, 0.42)',
 'ClassRoom(オトナのキャッシュレス勉強会(JP_Stripes 熊本 #1), 23, 5, 0.21739130434782608)',
 'ClassRoom(ROS Japan UG #29 ROS2講習会, 50, 0